In [1]:
import pandas as pd
import os, sys
import numpy as np
from hexmap.plot_summary import *

class HeightPlotter:
    
    def __init__(self, heights, xy, name, saveDir, label, hb_type = "LD"):
        self.heights = heights
        self.xy = xy
        self.name = name
        self.saveDir = saveDir
        self.label = label
        self.hb_type = hb_type
        
    
    def coor_transform(self, x, y, hb_type = "LD"):
        #max size of plot
        plot_x = 5.5
        if hb_type == "HD": plot_x = 5.9

        max_y = plot_x
        max_x = plot_x*np.tan(np.pi/6)
        #max size in real life
        max_y2 = 83.32
        max_x2 = 83.32*np.tan(np.pi/6)

        x_ = x*(max_x/max_x2)
        y_ = y*(max_y/max_y2)

        return x_, y_
    
    def pointToPad(self, hb_type):
    # To add the pad - channel and geometry mapping to the data DataFrame
    # df: pandas DataFrame with the data
    # hb_type: the type of the board ("LD" for low density or "HD" for high density)
        # create dataFrame clone to avoid conflict
        s = ""
        if not os.getcwd().endswith('hexmap'):
            s = 'hexmap/' 
        # geometry mapping files' paths
        ld_board_geo = s + "geometries/hex_positions_HPK_198ch_8inch_edge_ring_testcap.txt"
        hd_board_geo = s + "geometries/hex_positions_HPK_432ch_8inch_edge_ring_testcap.txt"

        # pad - channel mapping files' paths
        ld_board_chan = s + "channel_maps/ld_pad_to_channel_mapping.csv"
        hd_board_chan = s + "channel_maps/hd_pad_to_channel_mapping.csv"
        
        # import mapping files to pandas dataFrames and transform to python dicts
        if hb_type == "LD":
            chan_map_fname = ld_board_chan
            geo_fname = ld_board_geo
        elif hb_type == "HD":
            chan_map_fname = hd_board_chan
            geo_fname = hd_board_geo
            
        df_ch_map = pd.read_csv(chan_map_fname)
            
        df_pad_map = pd.read_csv(geo_fname, skiprows= 7, delim_whitespace=True, names = ['padnumber', 'xposition', 'yposition', 'type', 'optional'])
        df_pad_map = df_pad_map[["padnumber","xposition","yposition"]].set_index("padnumber")
        d_pad_map = df_pad_map.to_dict()
        
        pad_list = []
        chip_list = []
        channel_list = []
        channeltype_list = []
        for x in self.xy:
            x1, y1 = self.coor_transform(x[0], x[1], hb_type)
            p = 1
            r = 9999.
            #print('(x, y) = ({0}, {1}), (x1, y1) = ({2}, {3})'.format(x[0], x[1], x1, y1))
            for i, (x2, y2) in enumerate(zip(d_pad_map['xposition'], d_pad_map['yposition'])):
                dist = np.sqrt((x1 - d_pad_map['xposition'][x2])**2 + (y1 - d_pad_map['yposition'][y2])**2)
                if r > dist and x2 >= 0 and y2 >= 0 and ((hb_type == "LD" and x2 <= 198) or (hb_type == "HD" and x2 <= 444)):
                    #print('(x2, y2) = ({0}, {1}), dist = {2}, pad---> {3} == {4}'.format(d_pad_map['xposition'][x2], d_pad_map['yposition'][y2], dist, x2, y2))
                    r = dist
                    p = x2
            pad_list.append(p)
            idx = df_ch_map.index[df_ch_map["PAD"] == p].tolist()[0]
            chip_list.append(df_ch_map["ASIC"][idx])
            channel_list.append(df_ch_map["Channel"][idx])
            channeltype_list.append(df_ch_map["Channeltype"][idx])
            

        return pad_list, chip_list, channel_list, channeltype_list

    def padMapping(self):
        pad, chip, chan, chantype = self.pointToPad(self.hb_type)
        dfOut = {'Height': self.heights, 'chip': chip, 'channel': chan, 'channeltype': chantype}
        df = pd.DataFrame(dfOut)
        
        make_hexmap_plots_from_df(df, self.saveDir, hb_type = "LD", label = self.label)

ModuleNotFoundError: No module named 'hexmap'